In [31]:
import paramiko
import pickle
import os
import pandas as pd
import zipfile
from datetime import datetime,timedelta
pd.set_option('display.max_columns',110)
pd.set_option('display.max_rows',None)

In [32]:
hostname = "sftp.datashop.livevol.com"
port = 22
username = "nan2_lehigh_edu"
password = "PAIndex2023!"
remote_folder = "/subscriptions/order_000046197/item_000053507"
local_folder = os.path.join(os.getcwd(),"input")

In [33]:
def extract_zip(zip_filename, extraction_folder):
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        # Extract all files in the ZIP archive to the extraction folder
        zip_ref.extractall(extraction_folder)

In [34]:
def download_most_recent_zip(hostname, port, username, password, remote_folder, local_folder):
    # Create an SSH client
    ssh = paramiko.SSHClient()

    try:
        # Automatically add the server's host key
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

        # Connect to the SSH server
        ssh.connect(hostname, port, username, password)

        # Open an SFTP session
        with ssh.open_sftp() as sftp:
            # Change to the remote folder
            sftp.chdir(remote_folder)

            # List files in the remote folder
            files = sftp.listdir()

            # Find the most recent ZIP file with "UnderlyingEOD" in the name
            most_recent_zip = None
            most_recent_date = datetime(1970, 1, 1)  # Initialize with a very old date

            for file in files:
                if file.startswith("UnderlyingEOD") and not "Summaries" in file:
                    try:
                        file_date = datetime.strptime(file.split("_")[-1].split(".")[0], "%Y-%m-%d")
                        if file_date > most_recent_date:
                            most_recent_date = file_date
                            most_recent_zip = file
                    except ValueError:
                        pass  # Ignore files with date parsing issues

            if most_recent_zip:
                # Replace backslashes with forward slashes in file paths
                remote_path = os.path.join(remote_folder, most_recent_zip).replace("\\", "/")
                local_path = os.path.join(local_folder, most_recent_zip).replace("\\", "/")

                # Download the most recent ZIP file
                sftp.get(remote_path, local_path)

                # Extract files from the downloaded ZIP file
                extract_zip(local_path, local_folder)

                # Clean up: Remove downloaded ZIP file
                os.remove(local_path)

    except Exception as e:
        print(f"Error: {e}")

    finally:
        # Close the SSH connection
        ssh.close()

In [35]:
# SFTP server details
sftp_hostname = "sftp.datashop.livevol.com"
sftp_port = 22  # Change the port if your SFTP server uses a different port
sftp_username = "nan2_lehigh_edu"
sftp_password = "PAIndex2023!"
sftp_remote_folder = "/subscriptions/order_000046197/item_000053507"  # Change to the actual path on the SFTP server
local_download_folder = os.path.join(os.getcwd(), "input")  # Set to "input" folder inside the current directory

# Create the local download folder if it doesn't exist
os.makedirs(local_download_folder, exist_ok=True)

# Download and extract the most recent ZIP file
download_most_recent_zip(sftp_hostname, sftp_port, sftp_username, sftp_password, sftp_remote_folder, local_download_folder)

In [36]:
input_file_path = os.path.join(os.getcwd(), "input/date_dataframes.pkl")

with open(input_file_path, 'rb') as input_file:
    date_dataframes = pickle.load(input_file)

In [37]:
#date_dataframes

In [38]:
local_download_folder = os.path.join(os.getcwd(), "input")  # Set to "input" folder inside the current directory

# List all CSV files in the "input" folder
csv_files = [file for file in os.listdir(local_download_folder) if file.startswith("UnderlyingEOD_") and file.endswith(".csv")]
sorted_csv_files = sorted(csv_files, key=lambda x: datetime.strptime(x.split("_")[-1].split(".")[0], "%Y-%m-%d"), reverse=True)

if sorted_csv_files:
    most_recent_csv = sorted_csv_files[0]
    csv_file_path = os.path.join(local_download_folder, most_recent_csv)

    date_format = most_recent_csv.split("_")[-1].split(".")[0]

    eod_df = pd.read_csv(csv_file_path)

    date_dataframes[date_format] = eod_df

    try:
        os.remove(csv_file_path)
        print(f"Deleted: {csv_file_path}")
    except FileNotFoundError:
        print(f"File not found: {csv_file_path}")
    except Exception as e:
        print(f"Error deleting file: {e}")
else:
    print("No CSV files found in the 'input' folder.")


Deleted: C:\Users\micha\OneDrive\Documents\Projects\PA100\PA-Index\input\UnderlyingEOD_2024-07-05.csv


In [39]:
start_2023 = '2023-11-10'
end_2023 = '2024-06-28'
start_2024 = '2024-06-28'

In [154]:
import warnings

df = pd.read_excel('input/Ray-50-States.xlsx')
df = df.sort_values('Market Cap',ascending=False)
df.columns = df.columns.str.rstrip('\n')
df['Ticker'] = df['Ticker'].str.split(' ',n=1,expand=True)[0].replace(' ','')
ca_df = df[df['State of Domicile'] == 'NJ']
float_df = ca_df[['Ticker','Equity Float']].head(100)
float_df = float_df[float_df['Ticker'] != 'HEI/A']
float_df = float_df[float_df['Ticker'] != 'LEN/B']
float_df = float_df[float_df['Ticker'] != 'CWEN/A']
float_df = float_df[float_df['Ticker'] != 'AIOT']
float_df

eod_market_cap_pivot2023 = pd.DataFrame()

unique_dates = list(date_dataframes.keys())

for date in unique_dates:

    if date >= start_2023 and date <= end_2023:
    
        eod_df = date_dataframes[date]

        merged_df = pd.merge(float_df,eod_df,left_on='Ticker',right_on='underlying_symbol')

        merged_df['Market Cap'] = merged_df['Equity Float'] * merged_df['close']
    
        eod_market_cap_daily = merged_df.pivot_table(values='Market Cap',index='quote_date',columns='Ticker',aggfunc='sum')

        eod_market_cap_pivot2023 = pd.concat([eod_market_cap_pivot2023,eod_market_cap_daily])

eod_market_cap_pivot2023 = eod_market_cap_pivot2023.rename_axis(index='Date')
eod_market_cap_pivot2023.index = pd.to_datetime(eod_market_cap_pivot2023.index,errors='coerce')


eod_market_cap_pivot2023['mkt_cap_deleted_stock'] = 0

for ticker in float_df['Ticker']:
    last_valid_market_cap = None
    first_deletion = False

    for date_index, date in enumerate(eod_market_cap_pivot2023.index):

        if date != eod_market_cap_pivot2023.index[-1]:
        
            next_date = eod_market_cap_pivot2023.index[date_index + 1]
        
            market_cap = eod_market_cap_pivot2023.at[date,ticker]

            market_cap_next_date = eod_market_cap_pivot2023.at[next_date,ticker]

            if (pd.isna(market_cap) or market_cap == 0) and (pd.isna(market_cap_next_date) or market_cap_next_date == 0):
                if not first_deletion and last_valid_market_cap is not None:
                    eod_market_cap_pivot2023.at[date, 'mkt_cap_deleted_stock'] = last_valid_market_cap
                    first_deletion = True

            if (pd.isna(market_cap) or market_cap == 0) and market_cap_next_date != 0:
                if not first_deletion and last_valid_market_cap is not None:
                    eod_market_cap_pivot2023.at[date,ticker] = last_valid_market_cap
                    first_deletion = True
            
            else:
                last_valid_market_cap = market_cap
                first_deletion = False

        else:
            market_cap = eod_market_cap_pivot2023.at[date,ticker]
            if pd.isna(market_cap) or market_cap == 0:
                if not first_deletion and last_valid_market_cap is not None:
                    eod_market_cap_pivot2023.at[date,ticker] = last_valid_market_cap
                    first_deletion = True
            else:
                last_valid_market_cap = market_cap
                first_deletion = False

eod_market_cap_pivot2023["close_mkt_cap"] = eod_market_cap_pivot2023.drop(columns='mkt_cap_deleted_stock').sum(axis=1)


eod_market_cap_pivot2023["adj_mkt_cap"] = 0
eod_market_cap_pivot2023["divisor"] = 0
eod_market_cap_pivot2023["gross_index_level"] = 0
eod_market_cap_pivot2023["Index Value"] = 0


for i in range(0, len(eod_market_cap_pivot2023)):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning)
        if i == 0:
            eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "adj_mkt_cap"] = float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "close_mkt_cap"]) + float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "mkt_cap_deleted_stock"])
            eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "divisor"] = 1
            eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "gross_index_level"] = float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "close_mkt_cap"]) / float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "divisor"])
            eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "Index Value"] = float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "gross_index_level"]) / (float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[0], "close_mkt_cap"]) / 100)
        else:
            eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "adj_mkt_cap"] = float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i-1], "close_mkt_cap"]) - float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "mkt_cap_deleted_stock"])
            eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "divisor"] = (float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "adj_mkt_cap"]) / float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i-1], "close_mkt_cap"])) * float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i-1], "divisor"])
            eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "gross_index_level"] = float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "close_mkt_cap"]) / float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "divisor"])
            eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "Index Value"] = float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "gross_index_level"]) / (float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[0], "close_mkt_cap"]) / 100)

C:\Users\micha\AppData\Local\Temp\ipykernel_31840\1109835493.py:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '86612132.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  eod_market_cap_pivot2023.at[date, 'mkt_cap_deleted_stock'] = last_valid_market_cap


In [155]:
eod_market_cap_pivot2023

Ticker,ADMA,ADP,ALX,AQST,ASIX,ATEX,AWK,BCBP,BDX,BELFA,BELFB,BGS,BLFY,BPRN,BURL,CAR,CERT,CHD,CLBK,CLDX,CLMB,CNDT,CNOB,CPB,CRMD,CTLT,CTSH,CVLT,CWEN,DGX,DNB,EMBC,EPRT,EVGO,FRBA,FRPT,GNE,HAYW,HOUS,HOV,IART,IDT,INOD,INSM,JJSF,JNJ,KRNY,KVUE,LINC,MAMA,MG,MOV,MRK,MSEX,NFBK,NJR,OCFC,OGN,OLED,OTLK,PAHC,PBF,PCRX,PEG,PFS,PGC,PKBK,PRU,PTCT,SIGI,UMH,UNTY,VLGEA,VLY,VRE,VRSK,WH,WLY,ZTS,TVGN,mkt_cap_deleted_stock,close_mkt_cap,adj_mkt_cap,divisor,gross_index_level,Index Value
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-11-10,8.408215e+08,9.311542e+10,3.872242e+08,1.300932e+08,6.291538e+08,3.731287e+08,2.402901e+10,1.565677e+08,6.771994e+10,8.166287e+07,5.242435e+08,6.390075e+08,1.667254e+08,1.464573e+08,8.298971e+09,3.120224e+09,1.615392e+09,2.189941e+10,4.318510e+08,1.770891e+09,1.758685e+08,4.884930e+08,6.265537e+08,7.975596e+09,1.741745e+08,5.895364e+09,3.272432e+10,3.013263e+09,1.747768e+09,1.469018e+10,3.028924e+09,8.397869e+08,3.942658e+09,2.502969e+08,2.739943e+08,3.181806e+09,5.052614e+08,1.554050e+09,4.305283e+08,3.688326e+08,2.448388e+09,5.775460e+08,2.230059e+08,3.818575e+09,1.893072e+09,3.541247e+11,4.061832e+08,3.666051e+10,2.446815e+08,1.171421e+08,1.193865e+08,4.163666e+08,2.557757e+11,1.078309e+09,3.508408e+08,4.011853e+09,7.471304e+08,2.851254e+09,6.836444e+09,5.577680e+06,1.955960e+08,4.333367e+09,1.280498e+09,3.065658e+10,1.796225e+09,3.836017e+08,1.783561e+08,3.294903e+10,1.400157e+09,6.081071e+09,8.900557e+08,1.771426e+08,2.064368e+08,3.390663e+09,1.113899e+09,3.385723e+10,6.045374e+09,1.227615e+09,7.102734e+10,NaN,0.00,1.177861e+12,1.177861e+12,1.00000,1.177861e+12,100.000000
2023-11-13,8.385853e+08,9.224332e+10,3.853272e+08,1.293188e+08,6.286419e+08,3.578304e+08,2.369051e+10,1.586356e+08,6.805183e+10,8.062512e+07,5.238357e+08,6.213638e+08,1.679498e+08,1.502506e+08,7.968181e+09,3.100115e+09,1.556802e+09,2.219472e+10,4.378490e+08,1.769576e+09,1.821364e+08,4.816814e+08,6.212349e+08,7.975596e+09,1.684949e+08,5.816314e+09,3.260023e+10,3.019688e+09,1.737999e+09,1.463709e+10,2.996702e+09,8.403543e+08,3.881841e+09,2.751279e+08,2.742265e+08,3.200796e+09,4.983857e+08,1.522884e+09,4.177824e+08,3.672665e+08,2.478874e+09,5.814414e+08,2.200646e+08,3.839881e+09,1.888956e+09,3.550145e+11,4.028077e+08,3.656479e+10,2.386025e+08,1.154875e+08,1.255344e+08,4.146993e+08,2.583733e+11,1.086360e+09,3.508408e+08,4.027498e+09,7.482632e+08,2.833290e+09,6.752103e+09,5.647998e+06,1.976187e+08,4.390697e+09,1.263419e+09,3.051728e+10,1.805156e+09,3.865705e+08,1.778436e+08,3.275570e+10,1.400157e+09,6.093576e+09,8.649060e+08,1.802331e+08,2.076556e+08,3.352084e+09,1.058882e+09,3.394262e+10,6.006758e+09,1.232540e+09,7.079253e+10,NaN,0.00,1.179260e+12,1.177861e+12,1.00000,1.179260e+12,100.118815
2023-11-14,8.654200e+08,9.390195e+10,3.973750e+08,1.254470e+08,6.644765e+08,3.904170e+08,2.524096e+10,1.702305e+08,6.734477e+10,8.349984e+07,5.499307e+08,6.911714e+08,1.783574e+08,1.556629e+08,8.396748e+09,3.221767e+09,1.731375e+09,2.239729e+10,4.761024e+08,1.832682e+09,1.902887e+08,5.159343e+08,6.737136e+08,8.005187e+09,1.690358e+08,6.243183e+09,3.344402e+10,3.072800e+09,1.877202e+09,1.479413e+10,3.141703e+09,9.010686e+08,4.046915e+09,3.108846e+08,2.921057e+08,3.307616e+09,5.315142e+08,1.634798e+09,4.824560e+08,4.052103e+08,2.655356e+09,5.904623e+08,2.371777e+08,3.942792e+09,1.923300e+09,3.550866e+11,4.405006e+08,3.713910e+10,2.434151e+08,1.161493e+08,1.328721e+08,4.357677e+08,2.577428e+11,1.163895e+09,3.823315e+08,4.199588e+09,8.219000e+08,2.939795e+09,7.076508e+09,6.182421e+06,2.059118e+08,4.499427e+09,1.263868e+09,3.146752e+10,1.983646e+09,4.118836e+08,1.892215e+08,3.375459e+10,1.534584e+09,6.126327e+09,8.912825e+08,1.934782e+08,2.126170e+08,3.712155e+09,1.111507e+09,3.387431e+10,6.110785e+09,1.287128e+09,7.237753e+10,NaN,0.00,1.191340e+12,1.179260e+12,1.00000,1.191340e+12,101.144356
2023-11-15,8.497664e+08,9.392232e+10,3.868610e+08,1.231239e+08,6.752269e+08,3.988746e+08,2.537908e+10,1.723722e+08,6.749484e+10,8.

In [156]:
last_value = eod_market_cap_pivot2023.iloc[-1]['Index Value']

# "#############################################"

In [157]:
import warnings

df = pd.read_excel('input/Ray-50-States.xlsx')
df = df.sort_values('Market Cap',ascending=False)
df.columns = df.columns.str.rstrip('\n')
df['Ticker'] = df['Ticker'].str.split(' ',n=1,expand=True)[0].replace(' ','')
ca_df = df[df['State of Domicile'] == 'NJ']
float_df = ca_df[['Ticker','Equity Float']].head(100)
float_df = float_df[float_df['Ticker'] != 'HEI/A']
float_df = float_df[float_df['Ticker'] != 'LEN/B']
float_df = float_df[float_df['Ticker'] != 'CWEN/A']
float_df = float_df[float_df['Ticker'] != 'AIOT']
float_df

eod_market_cap_pivot2024 = pd.DataFrame()

unique_dates = list(date_dataframes.keys())

for date in unique_dates:

    if date >= start_2024:
    
        eod_df = date_dataframes[date]

        merged_df = pd.merge(float_df,eod_df,left_on='Ticker',right_on='underlying_symbol')

        merged_df['Market Cap'] = merged_df['Equity Float'] * merged_df['close']
    
        eod_market_cap_daily = merged_df.pivot_table(values='Market Cap',index='quote_date',columns='Ticker',aggfunc='sum')

        eod_market_cap_pivot2024 = pd.concat([eod_market_cap_pivot2024,eod_market_cap_daily])

eod_market_cap_pivot2024 = eod_market_cap_pivot2024.rename_axis(index='Date')
eod_market_cap_pivot2024.index = pd.to_datetime(eod_market_cap_pivot2024.index,errors='coerce')


eod_market_cap_pivot2024['mkt_cap_deleted_stock'] = 0

for ticker in float_df['Ticker']:
    last_valid_market_cap = None
    first_deletion = False

    for date_index, date in enumerate(eod_market_cap_pivot2024.index):

        if date != eod_market_cap_pivot2024.index[-1]:
        
            next_date = eod_market_cap_pivot2024.index[date_index + 1]
        
            market_cap = eod_market_cap_pivot2024.at[date,ticker]

            market_cap_next_date = eod_market_cap_pivot2024.at[next_date,ticker]

            if (pd.isna(market_cap) or market_cap == 0) and (pd.isna(market_cap_next_date) or market_cap_next_date == 0):
                if not first_deletion and last_valid_market_cap is not None:
                    eod_market_cap_pivot2024.at[date, 'mkt_cap_deleted_stock'] = last_valid_market_cap
                    first_deletion = True

            if (pd.isna(market_cap) or market_cap == 0) and market_cap_next_date != 0:
                if not first_deletion and last_valid_market_cap is not None:
                    eod_market_cap_pivot2024.at[date,ticker] = last_valid_market_cap
                    first_deletion = True
            
            else:
                last_valid_market_cap = market_cap
                first_deletion = False

        else:
            market_cap = eod_market_cap_pivot2024.at[date,ticker]
            if pd.isna(market_cap) or market_cap == 0:
                if not first_deletion and last_valid_market_cap is not None:
                    eod_market_cap_pivot2024.at[date,ticker] = last_valid_market_cap
                    first_deletion = True
            else:
                last_valid_market_cap = market_cap
                first_deletion = False

eod_market_cap_pivot2024["close_mkt_cap"] = eod_market_cap_pivot2024.drop(columns='mkt_cap_deleted_stock').sum(axis=1)


eod_market_cap_pivot2024["adj_mkt_cap"] = 0
eod_market_cap_pivot2024["divisor"] = 0
eod_market_cap_pivot2024["gross_index_level"] = 0
eod_market_cap_pivot2024["Index Value"] = 0


for i in range(0, len(eod_market_cap_pivot2024)):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning)
        if i == 0:
            eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "adj_mkt_cap"] = float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "close_mkt_cap"]) + float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "mkt_cap_deleted_stock"])
            eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "divisor"] = 1
            eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "gross_index_level"] = float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "close_mkt_cap"]) / float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "divisor"])
            eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "Index Value"] = float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "gross_index_level"]) / (float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[0], "close_mkt_cap"]) / last_value)
        else:
            eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "adj_mkt_cap"] = float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i-1], "close_mkt_cap"]) - float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "mkt_cap_deleted_stock"])
            eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "divisor"] = (float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "adj_mkt_cap"]) / float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i-1], "close_mkt_cap"])) * float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i-1], "divisor"])
            eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "gross_index_level"] = float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "close_mkt_cap"]) / float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "divisor"])
            eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "Index Value"] = float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "gross_index_level"]) / (float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[0], "close_mkt_cap"]) / last_value)

In [158]:
eod_market_cap_pivot2024.iloc[1:]

Ticker,ADMA,ADP,ALX,AQST,ASIX,ATEX,AWK,BCBP,BDX,BELFA,BELFB,BGS,BLFY,BPRN,BURL,CAR,CERT,CHD,CLBK,CLDX,CLMB,CNDT,CNOB,CPB,CRMD,CTLT,CTSH,CVLT,CWEN,DGX,DNB,EMBC,EPRT,EVGO,FRBA,FRPT,GNE,HAYW,HOUS,HOV,IART,IDT,INOD,INSM,JJSF,JNJ,KRNY,KVUE,LINC,MAMA,MG,MOV,MRK,MSEX,NFBK,NJR,OCFC,OGN,OLED,OTLK,PAHC,PBF,PCRX,PEG,PFS,PGC,PKBK,PRU,PTCT,SIGI,TVGN,UMH,UNTY,VLGEA,VLY,VRE,VRSK,WH,WLY,ZTS,mkt_cap_deleted_stock,close_mkt_cap,adj_mkt_cap,divisor,gross_index_level,Index Value
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-07-01,2.580606e+09,9.600478e+10,4.351530e+08,1.893321e+08,5.649074e+08,4.883011e+08,2.484605e+10,1.524320e+08,6.615285e+10,1.312513e+08,661038099.0,6.106242e+08,1.852957e+08,1.494179e+08,1.479478e+10,1.683342e+09,1.630936e+09,2.559202e+10,3.934643e+08,2.562993e+09,2.554866e+08,6.373764e+08,6.758411e+08,8.928407e+09,2.312410e+08,9.868937e+09,3.361775e+10,5.212281e+09,1.978958e+09,1.516016e+10,2.938701e+09,6.899870e+08,4.769765e+09,2.378814e+08,2.951243e+08,6.124335e+09,3.098242e+08,1.718380e+09,3.134548e+08,619270108.0,1.962640e+09,7.356176e+08,4.120529e+08,1.066419e+10,1.942002e+09,3.521286e+11,3.369858e+08,3.436324e+10,297113275.5,213767730.8,1.608347e+08,3.718046e+08,3.225063e+11,9.232399e+08,3.682283e+08,4.157543e+09,8.768444e+08,5.207196e+09,9.168119e+09,1.039310e+08,343860632.0,4.595307e+09,1.279149e+09,3.663636e+10,1.805030e+09,3.507885e+08,167080705.0,4.207867e+10,2.352475e+09,5.612426e+09,5252456.097,9.839072e+08,2.069014e+08,2.253637e+08,2.949148e+09,1.192040e+09,3.817638e+10,5.715168e+09,1.670065e+09,7.231463e+10,0,1.299953e+12,1.293311e+12,1,1.299953e+12,110.373281
2024-07-05,2.562717e+09,9.598032e+10,4.497233e+08,1.912680e+08,5.441745e+08,4.802166e+08,2.529738e+10,1.484439e+08,6.574592e+10,1.261424e+08,647175156.6,5.983504e+08,1.829897e+08,1.515921e+08,1.497129e+10,1.748323e+09,1.700286e+09,2.601912e+10,3.857336e+08,2.332922e+09,2.375639e+08,6.354302e+08,6.446376e+08,8.902762e+09,2.228568e+08,9.969067e+09,3.375672e+10,5.309082e+09,2.000937e+09,1.553173e+10,2.946757e+09,6.502674e+08,4.865334e+09,2.781077e+08,2.849076e+08,6.338924e+09,3.125328e+08,1.708463e+09,3.049575e+08,611126527.1,1.952478e+09,7.315172e+08,4.002876e+08,1.125272e+10,1.950470e+09,3.522488e+11,3.226400e+08,3.505242e+10,297113275.5,255462365.6,1.572650e+08,3.677122e+08,3.187991e+11,9.449426e+08,3.585686e+08,4.133099e+09,8.706136e+08,5.159718e+09,9.589823e+09,1.025247e+08,341332245.0,4.293829e+09,9.465527e+08,3.685556e+10,1.789307e+09,3.350070e+08,158880425.0,4.183521e+10,2.335936e+09,5.449263e+09,4950477.000,9.955620e+08,1.927661e+08,2.161363e+08,2.811979e+09,1.180080e+09,3.905586e+10,5.700982e+09,1.777189e+09,7.336710e+10,0,1.300367e+12,1.299953e+12,1,1.300367e+12,110.408434


In [159]:
eod_market_cap_pivot = pd.DataFrame()
eod_market_cap_pivot = pd.concat([eod_market_cap_pivot2023,eod_market_cap_pivot2024.iloc[1:]])
eod_market_cap_pivot

Ticker,ADMA,ADP,ALX,AQST,ASIX,ATEX,AWK,BCBP,BDX,BELFA,BELFB,BGS,BLFY,BPRN,BURL,CAR,CERT,CHD,CLBK,CLDX,CLMB,CNDT,CNOB,CPB,CRMD,CTLT,CTSH,CVLT,CWEN,DGX,DNB,EMBC,EPRT,EVGO,FRBA,FRPT,GNE,HAYW,HOUS,HOV,IART,IDT,INOD,INSM,JJSF,JNJ,KRNY,KVUE,LINC,MAMA,MG,MOV,MRK,MSEX,NFBK,NJR,OCFC,OGN,OLED,OTLK,PAHC,PBF,PCRX,PEG,PFS,PGC,PKBK,PRU,PTCT,SIGI,UMH,UNTY,VLGEA,VLY,VRE,VRSK,WH,WLY,ZTS,TVGN,mkt_cap_deleted_stock,close_mkt_cap,adj_mkt_cap,divisor,gross_index_level,Index Value
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-11-10,8.408215e+08,9.311542e+10,3.872242e+08,1.300932e+08,6.291538e+08,3.731287e+08,2.402901e+10,1.565677e+08,6.771994e+10,8.166287e+07,5.242435e+08,6.390075e+08,1.667254e+08,1.464573e+08,8.298971e+09,3.120224e+09,1.615392e+09,2.189941e+10,4.318510e+08,1.770891e+09,1.758685e+08,4.884930e+08,6.265537e+08,7.975596e+09,1.741745e+08,5.895364e+09,3.272432e+10,3.013263e+09,1.747768e+09,1.469018e+10,3.028924e+09,8.397869e+08,3.942658e+09,2.502969e+08,2.739943e+08,3.181806e+09,5.052614e+08,1.554050e+09,4.305283e+08,3.688326e+08,2.448388e+09,5.775460e+08,2.230059e+08,3.818575e+09,1.893072e+09,3.541247e+11,4.061832e+08,3.666051e+10,2.446815e+08,1.171421e+08,1.193865e+08,4.163666e+08,2.557757e+11,1.078309e+09,3.508408e+08,4.011853e+09,7.471304e+08,2.851254e+09,6.836444e+09,5.577680e+06,1.955960e+08,4.333367e+09,1.280498e+09,3.065658e+10,1.796225e+09,3.836017e+08,1.783561e+08,3.294903e+10,1.400157e+09,6.081071e+09,8.900557e+08,1.771426e+08,2.064368e+08,3.390663e+09,1.113899e+09,3.385723e+10,6.045374e+09,1.227615e+09,7.102734e+10,NaN,0.00,1.177861e+12,1.177861e+12,1.00000,1.177861e+12,100.000000
2023-11-13,8.385853e+08,9.224332e+10,3.853272e+08,1.293188e+08,6.286419e+08,3.578304e+08,2.369051e+10,1.586356e+08,6.805183e+10,8.062512e+07,5.238357e+08,6.213638e+08,1.679498e+08,1.502506e+08,7.968181e+09,3.100115e+09,1.556802e+09,2.219472e+10,4.378490e+08,1.769576e+09,1.821364e+08,4.816814e+08,6.212349e+08,7.975596e+09,1.684949e+08,5.816314e+09,3.260023e+10,3.019688e+09,1.737999e+09,1.463709e+10,2.996702e+09,8.403543e+08,3.881841e+09,2.751279e+08,2.742265e+08,3.200796e+09,4.983857e+08,1.522884e+09,4.177824e+08,3.672665e+08,2.478874e+09,5.814414e+08,2.200646e+08,3.839881e+09,1.888956e+09,3.550145e+11,4.028077e+08,3.656479e+10,2.386025e+08,1.154875e+08,1.255344e+08,4.146993e+08,2.583733e+11,1.086360e+09,3.508408e+08,4.027498e+09,7.482632e+08,2.833290e+09,6.752103e+09,5.647998e+06,1.976187e+08,4.390697e+09,1.263419e+09,3.051728e+10,1.805156e+09,3.865705e+08,1.778436e+08,3.275570e+10,1.400157e+09,6.093576e+09,8.649060e+08,1.802331e+08,2.076556e+08,3.352084e+09,1.058882e+09,3.394262e+10,6.006758e+09,1.232540e+09,7.079253e+10,NaN,0.00,1.179260e+12,1.177861e+12,1.00000,1.179260e+12,100.118815
2023-11-14,8.654200e+08,9.390195e+10,3.973750e+08,1.254470e+08,6.644765e+08,3.904170e+08,2.524096e+10,1.702305e+08,6.734477e+10,8.349984e+07,5.499307e+08,6.911714e+08,1.783574e+08,1.556629e+08,8.396748e+09,3.221767e+09,1.731375e+09,2.239729e+10,4.761024e+08,1.832682e+09,1.902887e+08,5.159343e+08,6.737136e+08,8.005187e+09,1.690358e+08,6.243183e+09,3.344402e+10,3.072800e+09,1.877202e+09,1.479413e+10,3.141703e+09,9.010686e+08,4.046915e+09,3.108846e+08,2.921057e+08,3.307616e+09,5.315142e+08,1.634798e+09,4.824560e+08,4.052103e+08,2.655356e+09,5.904623e+08,2.371777e+08,3.942792e+09,1.923300e+09,3.550866e+11,4.405006e+08,3.713910e+10,2.434151e+08,1.161493e+08,1.328721e+08,4.357677e+08,2.577428e+11,1.163895e+09,3.823315e+08,4.199588e+09,8.219000e+08,2.939795e+09,7.076508e+09,6.182421e+06,2.059118e+08,4.499427e+09,1.263868e+09,3.146752e+10,1.983646e+09,4.118836e+08,1.892215e+08,3.375459e+10,1.534584e+09,6.126327e+09,8.912825e+08,1.934782e+08,2.126170e+08,3.712155e+09,1.111507e+09,3.387431e+10,6.110785e+09,1.287128e+09,7.237753e+10,NaN,0.00,1.191340e+12,1.179260e+12,1.00000,1.191340e+12,101.144356
2023-11-15,8.497664e+08,9.392232e+10,3.868610e+08,1.231239e+08,6.752269e+08,3.988746e+08,2.537908e+10,1.723722e+08,6.749484e+10,8.

In [160]:
index_df = eod_market_cap_pivot[['Index Value']].copy()
index_df

Ticker,Index Value
Date,
2023-11-10,100.000000
2023-11-13,100.118815
2023-11-14,101.144356
2023-11-15,101.487855
2023-11-16,101.908836
2023-11-17,101.896981
2023-11-20,102.243278
2023-11-21,102.673601
2023-11-22,102.747221


In [17]:
# pathhh = os.path.join(os.getcwd(), 'input/input2023.xlsx')

In [18]:
# eod_market_cap_pivot.to_excel(pathhh, index=True)

In [20]:
unique_dates[:]

['2023-11-10',
 '2023-11-13',
 '2023-11-14',
 '2023-11-15',
 '2023-11-16',
 '2023-11-17',
 '2023-11-20',
 '2023-11-21',
 '2023-11-22',
 '2023-11-24',
 '2023-11-27',
 '2023-11-28',
 '2023-11-29',
 '2023-11-30',
 '2023-12-01',
 '2023-12-04',
 '2023-12-05',
 '2023-12-06',
 '2023-12-07',
 '2023-12-08',
 '2023-12-11',
 '2023-12-12',
 '2023-12-13',
 '2023-12-14',
 '2023-12-15',
 '2023-12-18',
 '2023-12-19',
 '2023-12-20',
 '2023-12-21',
 '2023-12-22',
 '2023-12-26',
 '2023-12-27',
 '2023-12-28',
 '2023-12-29',
 '2024-01-02',
 '2024-01-03',
 '2024-01-04',
 '2024-01-05',
 '2024-01-08',
 '2024-01-09',
 '2024-01-10',
 '2024-01-11',
 '2024-01-12',
 '2024-01-16',
 '2024-01-17',
 '2024-01-18',
 '2024-01-19',
 '2024-01-22',
 '2024-01-23',
 '2024-01-24',
 '2024-01-25',
 '2024-01-26',
 '2024-01-29',
 '2024-01-30',
 '2024-01-31',
 '2024-02-01',
 '2024-02-02',
 '2024-02-05',
 '2024-02-06',
 '2024-02-07',
 '2024-02-08',
 '2024-02-09',
 '2024-02-12',
 '2024-02-13',
 '2024-02-14',
 '2024-02-15',
 '2024-02-

In [27]:
    start_2023 = '2023-11-10'
    end_2023 = '2024-06-28'
    start_2024 = '2024-06-28'
    
    excel_file_path2023 = 'input/RAY as of Oct 23 20231_PA.xlsx'
    df2023 = pd.read_excel(excel_file_path2023)
    df2023 = df2023.sort_values('Market Cap\n',ascending=False)
    df2023.columns = df2023.columns.str.rstrip('\n')
    df2023['Ticker'] = df2023['Ticker'].str.split(' ',n=1,expand=True)[0].replace(' ','')
    float_df2023 = df2023[['Ticker','Equity Float']].head(100)
    
    eod_market_cap_pivot2023 = pd.DataFrame()

    excel_file_path2024 = 'input/RAY as of Jul 01 20241_PA.xlsx'
    df2024 = pd.read_excel(excel_file_path2024)
    df2024 = df2024.sort_values('Market Cap\n',ascending=False)
    df2024.columns = df2024.columns.str.rstrip('\n')
    df2024['Ticker'] = df2024['Ticker'].str.split(' ',n=1,expand=True)[0].replace(' ','')
    float_df2024 = df2024[['Ticker','Equity Float']].head(100)

    eod_market_cap_pivot2024 = pd.DataFrame()
    
    unique_dates = list(date_dataframes.keys())




    
    for date in unique_dates:

        if date >= start_2023 and date <= end_2023:
    
            eod_df2023 = date_dataframes[date]

            merged_df2023 = pd.merge(float_df2023,eod_df2023,left_on='Ticker',right_on='underlying_symbol')
    
            merged_df2023['Market Cap'] = merged_df2023['Equity Float'] * merged_df2023['close']
        
            eod_market_cap_daily2023 = merged_df2023.pivot_table(values='Market Cap',index='quote_date',columns='Ticker',aggfunc='sum')
    
            eod_market_cap_pivot2023 = pd.concat([eod_market_cap_pivot2023,eod_market_cap_daily2023])
    
    eod_market_cap_pivot2023 = eod_market_cap_pivot2023.rename_axis(index='Date')
    eod_market_cap_pivot2023.index = pd.to_datetime(eod_market_cap_pivot2023.index,errors='coerce')
    
    
    eod_market_cap_pivot2023['mkt_cap_deleted_stock'] = 0
    
    for ticker in float_df2023['Ticker']:
        last_valid_market_cap = None
        first_deletion = False
    
        for date_index, date in enumerate(eod_market_cap_pivot2023.index):
    
            if date != eod_market_cap_pivot2023.index[-1]:
            
                next_date = eod_market_cap_pivot2023.index[date_index + 1]
            
                market_cap = eod_market_cap_pivot2023.at[date,ticker]
    
                market_cap_next_date = eod_market_cap_pivot2023.at[next_date,ticker]
    
                if (pd.isna(market_cap) or market_cap == 0) and (pd.isna(market_cap_next_date) or market_cap_next_date == 0):
                    if not first_deletion and last_valid_market_cap is not None:
                        eod_market_cap_pivot2023.at[date, 'mkt_cap_deleted_stock'] = last_valid_market_cap
                        first_deletion = True
    
                if (pd.isna(market_cap) or market_cap == 0) and market_cap_next_date != 0:
                    if not first_deletion and last_valid_market_cap is not None:
                        eod_market_cap_pivot2023.at[date,ticker] = last_valid_market_cap
                        first_deletion = True
                
                else:
                    last_valid_market_cap = market_cap
                    first_deletion = False
    
            else:
                market_cap = eod_market_cap_pivot2023.at[date,ticker]
                if pd.isna(market_cap) or market_cap == 0:
                    if not first_deletion and last_valid_market_cap is not None:
                        eod_market_cap_pivot2023.at[date,ticker] = last_valid_market_cap
                        first_deletion = True
                else:
                    last_valid_market_cap = market_cap
                    first_deletion = False
    
    eod_market_cap_pivot2023["close_mkt_cap"] = eod_market_cap_pivot2023.drop(columns='mkt_cap_deleted_stock').sum(axis=1)
    
    
    eod_market_cap_pivot2023["adj_mkt_cap"] = 0
    eod_market_cap_pivot2023["divisor"] = 0
    eod_market_cap_pivot2023["gross_index_level"] = 0
    eod_market_cap_pivot2023["Index Value"] = 0
    
    
    for i in range(0, len(eod_market_cap_pivot2023)):
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=FutureWarning)
            if i == 0:
                eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "adj_mkt_cap"] = float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "close_mkt_cap"]) + float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "mkt_cap_deleted_stock"])
                eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "divisor"] = 1
                eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "gross_index_level"] = float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "close_mkt_cap"]) / float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "divisor"])
                eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "Index Value"] = float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "gross_index_level"]) / (float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[0], "close_mkt_cap"]) / 100)
            else:
                eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "adj_mkt_cap"] = float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i-1], "close_mkt_cap"]) - float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "mkt_cap_deleted_stock"])
                eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "divisor"] = (float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "adj_mkt_cap"]) / float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i-1], "close_mkt_cap"])) * float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i-1], "divisor"])
                eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "gross_index_level"] = float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "close_mkt_cap"]) / float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "divisor"])
                eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "Index Value"] = float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[i], "gross_index_level"]) / (float(eod_market_cap_pivot2023.at[eod_market_cap_pivot2023.index[0], "close_mkt_cap"]) / 100)


    for date in unique_dates:
    
        if date >= start_2024:
        
            eod_df2024 = date_dataframes[date]
    
            merged_df2024 = pd.merge(float_df2024,eod_df2024,left_on='Ticker',right_on='underlying_symbol')
    
            merged_df2024['Market Cap'] = merged_df2024['Equity Float'] * merged_df2024['close']
        
            eod_market_cap_daily2024 = merged_df2024.pivot_table(values='Market Cap',index='quote_date',columns='Ticker',aggfunc='sum')
    
            eod_market_cap_pivot2024 = pd.concat([eod_market_cap_pivot2024,eod_market_cap_daily2024])
    
    eod_market_cap_pivot2024 = eod_market_cap_pivot2024.rename_axis(index='Date')
    eod_market_cap_pivot2024.index = pd.to_datetime(eod_market_cap_pivot2024.index,errors='coerce')
    
    
    eod_market_cap_pivot2024['mkt_cap_deleted_stock'] = 0
    
    for ticker in float_df2024['Ticker']:
        last_valid_market_cap = None
        first_deletion = False
    
        for date_index, date in enumerate(eod_market_cap_pivot2024.index):
    
            if date != eod_market_cap_pivot2024.index[-1]:
            
                next_date = eod_market_cap_pivot2024.index[date_index + 1]
            
                market_cap = eod_market_cap_pivot2024.at[date,ticker]
    
                market_cap_next_date = eod_market_cap_pivot2024.at[next_date,ticker]
    
                if (pd.isna(market_cap) or market_cap == 0) and (pd.isna(market_cap_next_date) or market_cap_next_date == 0):
                    if not first_deletion and last_valid_market_cap is not None:
                        eod_market_cap_pivot2024.at[date, 'mkt_cap_deleted_stock'] = last_valid_market_cap
                        first_deletion = True
    
                if (pd.isna(market_cap) or market_cap == 0) and market_cap_next_date != 0:
                    if not first_deletion and last_valid_market_cap is not None:
                        eod_market_cap_pivot2024.at[date,ticker] = last_valid_market_cap
                        first_deletion = True
                
                else:
                    last_valid_market_cap = market_cap
                    first_deletion = False
    
            else:
                market_cap = eod_market_cap_pivot2024.at[date,ticker]
                if pd.isna(market_cap) or market_cap == 0:
                    if not first_deletion and last_valid_market_cap is not None:
                        eod_market_cap_pivot2024.at[date,ticker] = last_valid_market_cap
                        first_deletion = True
                else:
                    last_valid_market_cap = market_cap
                    first_deletion = False
    
    eod_market_cap_pivot2024["close_mkt_cap"] = eod_market_cap_pivot2024.drop(columns='mkt_cap_deleted_stock').sum(axis=1)
    
    
    eod_market_cap_pivot2024["adj_mkt_cap"] = 0
    eod_market_cap_pivot2024["divisor"] = 0
    eod_market_cap_pivot2024["gross_index_level"] = 0
    eod_market_cap_pivot2024["Index Value"] = 0
    
    
    for i in range(0, len(eod_market_cap_pivot2024)):
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=FutureWarning)
            if i == 0:
                eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "adj_mkt_cap"] = float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "close_mkt_cap"]) + float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "mkt_cap_deleted_stock"])
                eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "divisor"] = 1
                eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "gross_index_level"] = float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "close_mkt_cap"]) / float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "divisor"])
                eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "Index Value"] = float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "gross_index_level"]) / (float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[0], "close_mkt_cap"]) / last_value)
            else:
                eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "adj_mkt_cap"] = float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i-1], "close_mkt_cap"]) - float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "mkt_cap_deleted_stock"])
                eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "divisor"] = (float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "adj_mkt_cap"]) / float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i-1], "close_mkt_cap"])) * float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i-1], "divisor"])
                eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "gross_index_level"] = float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "close_mkt_cap"]) / float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "divisor"])
                eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "Index Value"] = float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[i], "gross_index_level"]) / (float(eod_market_cap_pivot2024.at[eod_market_cap_pivot2024.index[0], "close_mkt_cap"]) / last_value)



    eod_market_cap_pivot = pd.DataFrame()
    eod_market_cap_pivot = pd.concat([eod_market_cap_pivot2023,eod_market_cap_pivot2024.iloc[1:]])

eod_market_cap_pivot

index_df = eod_market_cap_pivot[["Index Value"]].copy()
index_df

C:\Users\micha\AppData\Local\Temp\ipykernel_28388\1773761835.py:65: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2938245904.56' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  eod_market_cap_pivot2023.at[date, 'mkt_cap_deleted_stock'] = last_valid_market_cap


Ticker,Index Value
Date,
2023-11-10,100.000000
2023-11-13,99.755815
2023-11-14,102.269793
2023-11-15,102.990948
2023-11-16,102.824421
2023-11-17,103.129197
2023-11-20,103.675161
2023-11-21,103.323209
2023-11-22,103.430614


In [28]:
    excel_file_path = 'input/RAY as of Oct 23 20231_PA.xlsx'
    df = pd.read_excel(excel_file_path)
    #df = pd.read_excel("/srv/paindex-test/data/RAY as of Oct 23 20231_PA.xlsx")
    df.columns = df.columns.str.rstrip('\n')
    df['Ticker'] = df['Ticker'].str.split(' ', n=1, expand=True)[0]
    df['Ticker'] = df['Ticker'].str.replace(' ', '')  # Remove spaces
    # Create float_df by selecting the first 100 rows of "Ticker" and "Equity Float" columns
    float_df = df[['Ticker', 'Equity Float']].head(100)
    float_df
    # Create an empty DataFrame to store market cap DataFrames for each date
    eod_market_cap_pivot = pd.DataFrame()

    # Extract unique dates from date_dataframes dictionary
    unique_dates = list(date_dataframes.keys())

    # Iterate over unique dates
    for date_format in unique_dates[:]:
        # Access the corresponding DataFrame from date_dataframes
        eod_df = date_dataframes[date_format]

        # Merge dataframes on the common column 'Ticker' and 'underlying_symbol'
        merged_df = pd.merge(float_df, eod_df, left_on='Ticker', right_on='underlying_symbol')

        # Perform the multiplication and rename the column to 'Market Cap'
        merged_df['Market Cap'] = merged_df['Equity Float'] * merged_df['close']

        # Use pivot_table to create a multi-level DataFrame
        eod_market_cap_daily = merged_df.pivot_table(index='quote_date', columns='Ticker', values='Market Cap', aggfunc='sum')

        # Add a new column for the sum of market caps for each date
        #eod_market_cap_daily['close index market cap'] = eod_market_cap_daily.sum(axis=1)

        # Append the daily market cap DataFrame to the main DataFrame
        eod_market_cap_pivot = pd.concat([eod_market_cap_pivot, eod_market_cap_daily])

    # Rename the index to 'Date'
    eod_market_cap_pivot = eod_market_cap_pivot.rename_axis(index='Date')

    # Display the resulting pivot table
    eod_market_cap_pivot.index = pd.to_datetime(eod_market_cap_pivot.index, errors='coerce')
    #eod_market_cap_pivot.index.name = None  # Remove the index name

    # Display the resulting pivot table
    eod_market_cap_pivot


    # Ensure the index is in DateTime format
    eod_market_cap_pivot.index = pd.to_datetime(eod_market_cap_pivot.index)
    # Assuming eod_market_cap_pivot is your DataFrame and float_df["Ticker"] is the DataFrame with Ticker information

    # Step 1: Calculate the close market cap
    eod_market_cap_pivot["close_market_cap"] = eod_market_cap_pivot.sum(axis=1)

    # Step 2: Calculate the gross change mkt cap
    eod_market_cap_pivot["gross_change_mkt_cap"] = eod_market_cap_pivot["close_market_cap"].diff().fillna(0)

    # Step 3: Calculate the mkt_cap_deleted_stock

#     mkt_cap_deleted_stock = {}
#     for date in eod_market_cap_pivot.index:
#         for ticker in float_df["Ticker"]:
#             if pd.isna(eod_market_cap_pivot.at[date, ticker]) or eod_market_cap_pivot.at[date, ticker] == 0:
#                 if ticker not in mkt_cap_deleted_stock:
#                     # Get the most recent non-zero market cap for the Ticker
#                     non_zero_caps = eod_market_cap_pivot.loc[:date, ticker].replace({0: pd.NA}).dropna()
#                     if not non_zero_caps.empty:
#                         mkt_cap_deleted_stock[ticker] = (non_zero_caps.iloc[-1], date)
#                     else:
#                         mkt_cap_deleted_stock[ticker] = (0, date)

#     # Prepare a column for the summed market caps
#     eod_market_cap_pivot["mkt_cap_deleted_stock"] = 0

#     # Populate the new column in the eod_market_cap_pivot dataframe
#     for ticker, (market_cap, cap_date) in mkt_cap_deleted_stock.items():
#         # This line assumes 'cap_date' matches an index exactly in eod_market_cap_pivot
#         if cap_date in eod_market_cap_pivot.index:
#             eod_market_cap_pivot.at[cap_date, "mkt_cap_deleted_stock"] += market_cap

#     # Make the values negative
#     eod_market_cap_pivot["mkt_cap_deleted_stock"] = -eod_market_cap_pivot["mkt_cap_deleted_stock"]

#     # Set mkt_cap_deleted_stock to 0 if it is NaN
#     eod_market_cap_pivot["mkt_cap_deleted_stock"] = eod_market_cap_pivot["mkt_cap_deleted_stock"].fillna(0)

# Initialization of the result column in the DataFrame
    eod_market_cap_pivot['mkt_cap_deleted_stock'] = 0

    # Iterate over each ticker
    for ticker in float_df['Ticker']:
        # Track the last valid market cap
        last_valid_market_cap = None
        # Flag to mark the first deletion occurrence
        first_deletion = False

        # Iterate through each date
        for date in eod_market_cap_pivot.index:
            market_cap = eod_market_cap_pivot.at[date, ticker]

            # Check if market cap is zero or NaN
            if pd.isna(market_cap) or market_cap == 0:
                if not first_deletion and last_valid_market_cap is not None:
                    # Mark the first deletion and update the DataFrame
                    eod_market_cap_pivot.at[date, 'mkt_cap_deleted_stock'] = -last_valid_market_cap
                    first_deletion = True
            else:
                # Reset on finding a non-zero market cap
                last_valid_market_cap = market_cap
                first_deletion = False  # Reset the deletion flag on valid data

    # Step 4: Calculate adj_mkt_cap
    #eod_market_cap_pivot["mkt_cap_deleted_stock"][4] = -50000000000
    #eod_market_cap_pivot["mkt_cap_deleted_stock"][4] = 5000000000
    eod_market_cap_pivot["adj_mkt_cap"] = eod_market_cap_pivot["close_market_cap"] + eod_market_cap_pivot["mkt_cap_deleted_stock"]

    # Step 5: Create gross_index_level column and set the first row value to be 100
    eod_market_cap_pivot["close_divisor"] = 0
    eod_market_cap_pivot["adj_divisor"] = 0
    eod_market_cap_pivot["gross_index_level"] = 0
    eod_market_cap_pivot["Index Value"] = 0
    # Temporarily filter out FutureWarnings
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning)
        eod_market_cap_pivot.at[eod_market_cap_pivot.index[0], "gross_index_level"] = 100
        eod_market_cap_pivot.at[eod_market_cap_pivot.index[0], "Index Value"] = 100
        eod_market_cap_pivot.at[eod_market_cap_pivot.index[0], "close_divisor"] = float(eod_market_cap_pivot["close_market_cap"][0]) / float(eod_market_cap_pivot["gross_index_level"][0])
        eod_market_cap_pivot.at[eod_market_cap_pivot.index[0], "adj_divisor"] = float(eod_market_cap_pivot["close_divisor"][0]) * float(eod_market_cap_pivot["close_market_cap"][0]) / float(eod_market_cap_pivot["adj_mkt_cap"][0])

    # Iterate through rows starting from the second row
    for i in range(1, len(eod_market_cap_pivot)):
            # Temporarily filter out FutureWarnings
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=FutureWarning)
            # Calculate close_divisor based on the previous day's adj_divisor
            eod_market_cap_pivot.at[eod_market_cap_pivot.index[i], "close_divisor"] = float(eod_market_cap_pivot.at[eod_market_cap_pivot.index[i-1], "adj_divisor"])

            # Calculate gross_index_level
            eod_market_cap_pivot.at[eod_market_cap_pivot.index[i], "gross_index_level"] = float(eod_market_cap_pivot.at[eod_market_cap_pivot.index[i], "close_market_cap"]) / float(eod_market_cap_pivot.at[eod_market_cap_pivot.index[i], "close_divisor"])

            # Calculate adj_divisor
            eod_market_cap_pivot.at[eod_market_cap_pivot.index[i], "adj_divisor"] = float(eod_market_cap_pivot.at[eod_market_cap_pivot.index[i], "adj_mkt_cap"]) / float(eod_market_cap_pivot.at[eod_market_cap_pivot.index[i], "close_market_cap"]) * float(eod_market_cap_pivot.at[eod_market_cap_pivot.index[i], "close_divisor"])

            # Calculate Index Value
            eod_market_cap_pivot.at[eod_market_cap_pivot.index[i], "Index Value"] = float(eod_market_cap_pivot.at[eod_market_cap_pivot.index[i], "adj_mkt_cap"]) / float(eod_market_cap_pivot.at[eod_market_cap_pivot.index[i], "adj_divisor"])

    eod_market_cap_pivot

index_df = eod_market_cap_pivot[["Index Value"]].copy()
index_df

C:\Users\micha\AppData\Local\Temp\ipykernel_28388\3057297699.py:104: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-243417757.38560003' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  eod_market_cap_pivot.at[date, 'mkt_cap_deleted_stock'] = -last_valid_market_cap


Ticker,Index Value
Date,
2023-11-10,100.000000
2023-11-13,99.794327
2023-11-14,102.140534
2023-11-15,102.912644
2023-11-16,102.783860
2023-11-17,103.065570
2023-11-20,103.647055
2023-11-21,103.322953
2023-11-22,103.442105


In [5]:
import streamlit as st
import numpy as np
import pandas as pd

chart_data = pd.DataFrame(
     np.random.randn(20, 3),
     columns=['a', 'b', 'c'])

st.line_chart(chart_data)

DeltaGenerator()